## Module 19 Challenge - Using deep learning model

In [48]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [49]:
# Questions to consider
#What variable(s) are considered the target for your model?---- IS SUCCESSFUL
#What variable(s) are considered to be the features for your model? 
#What variable(s) are neither and should be removed from the input data?

In [50]:
charity_df['ASK_AMT'].value_counts()

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [64]:
# Groups classification cases under 20
class_low = charity_df['CLASSIFICATION'].value_counts()

# Determine which values to replace
bucket_class = list(class_low[class_low < 20].index)

# Replace in DataFrame
for class_low in bucket_class:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(class_low,"Under 20")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000       17326
C2000        6074
C1200        4837
C3000        1918
C2100        1883
C7000         777
C1700         287
Under 20      207
C4000         194
C5000         116
C1270         114
C2700         104
C2800          95
C7100          75
C1300          58
C1280          50
C1230          36
C1400          34
C7200          32
C2300          32
C1240          30
C8000          20
Name: CLASSIFICATION, dtype: int64

In [78]:
# CHeck for bucketing potential
charity_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [79]:
# check for bucketing potential
charity_df["INCOME_AMT"].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [65]:
# Check data types 
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [81]:
# Drop variables that are neither features nor the target
charity = charity_df.drop(columns=['EIN','NAME', 'SPECIAL_CONSIDERATIONS'])

In [67]:
# Preprocessing - Encode all categorical data
# Generate our categorical variable list
charity_cat = charity.dtypes[charity.dtypes == "object"].index.tolist()
charity_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [68]:
# Check the number of unique values in each column
charity[charity_cat].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      22
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [69]:
# Create a OneHotEncoder instance
encode = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(encode.fit_transform(charity[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = encode.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# Merge one-hot encoded features and drop the originals
charity = charity.merge(encode_df,left_index=True, right_index=True)
charity = charity.drop(charity_cat,1)
charity.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# Split our preprocessed data into our features and target arrays
y = charity["IS_SUCCESSFUL"].values
X = charity.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [72]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [88]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 4

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 7)                 462       
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 5         
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [89]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [90]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
25724/25724 [==============================] - 2s 72us/sample - loss: 13225.9431 - acc: 0.5220
Epoch 2/100
25724/25724 [==============================] - 2s 60us/sample - loss: 2727.4624 - acc: 0.4986
Epoch 3/100
25724/25724 [==============================] - 2s 61us/sample - loss: 1568.9252 - acc: 0.5108
Epoch 4/100
25724/25724 [==============================] - 2s 60us/sample - loss: 727.1126 - acc: 0.5051
Epoch 5/100
25724/25724 [==============================] - 2s 61us/sample - loss: 399.7981 - acc: 0.5128
Epoch 6/100
25724/25724 [==============================] - 2s 63us/sample - loss: 2127.7848 - acc: 0.5235
Epoch 7/100
25724/25724 [==============================] - 2s 67us/sample - loss: 589.2257 - acc: 0.5153
Epoch 8/100
25724/25724 [==============================] - 2s 61us/sample - loss: 1310.5598 - acc: 0.5606
Epoch 9/100
25724/25724 [==============================] - 2s 61us/sample - loss: 656.3432 - acc: 0.5043
Epoch 10/100
25724/25724 [=======================

25724/25724 [==============================] - 2s 66us/sample - loss: 0.6911 - acc: 0.5321
Epoch 80/100
25724/25724 [==============================] - 2s 61us/sample - loss: 0.6911 - acc: 0.5321
Epoch 81/100
25724/25724 [==============================] - 2s 77us/sample - loss: 0.6911 - acc: 0.5321
Epoch 82/100
25724/25724 [==============================] - 2s 65us/sample - loss: 0.6911 - acc: 0.5321
Epoch 83/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.6911 - acc: 0.5321
Epoch 84/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.6911 - acc: 0.5321
Epoch 85/100
25724/25724 [==============================] - 1s 55us/sample - loss: 0.6911 - acc: 0.5321
Epoch 86/100
25724/25724 [==============================] - 1s 54us/sample - loss: 0.6911 - acc: 0.5321
Epoch 87/100
25724/25724 [==============================] - 1s 56us/sample - loss: 0.6911 - acc: 0.5321
Epoch 88/100
25724/25724 [==============================] - 1s 57us/sample - 

In [91]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6910 - acc: 0.5332
Loss: 0.6909715246945707, Accuracy: 0.5331778526306152
